Seasonal Recommendation Model

In [1]:
from enum import Enum

class YomiCycle(Enum):
    # [0] == hebcal API key; [1] == yutorah series_name; [2] == yutorah category
    DAF = ["F", "Daf Yomi", "category_Gemara"]
    WEEKLY_DAF = ["dw", "Daf Hashvua", "category_Gemara"]
    MISHNAH = ["myomi", "Mishna Yomi LZN Daniel Ari ben Avraham Kadesh", "category_Mishna"]
    PARSHA = ["s", 'Fifteen Minute Parsha', "category_Parsha"]
    NACH = ["nyomi", "Nach Yomi", "category_Nach"]
    YERUSHALMI = ["yyomi", "Yerushalmi Yomi", "category_Yerushalmi"]
    #AMUD = ""
    #MISHNAH_BERURAH = ""

In [6]:
# from src.pipeline.data_processor import DataProcessor, CleanedData
 
# dp = DataProcessor()
# shiurs = dp.load_table(CleanedData.SHIURIM)
# print(shiurs['series_name'].unique())

2024-07-22 10:38:16,846 - root - INFO - DataProcessor instance created
2024-07-22 10:38:16,849 - root - INFO - Loading data from: shiurim_cleaned


['Daily Shiur' '' 'Sichas Mussar' 'BCBM' 'Chassidus' 'Kollel Yom Rishon'
 'Gruss Kollel' 'Yoreh Deah' 'Torah uMadda Lecture Series' 'Daf Yomi'
 'Contemporary Halacha' 'Weekly Shiur  CBEY'
 'YU Student Medical Ethics Society' 'Weekly Shiur' 'Midreshet Yom Rishon'
 'MJE' 'Weekly Shiur  YIFH' 'YIJE' 'Ten Minute Halacha'
 'Yeshivat Torat Shraga' '4th Year Halacha Lemaaseh' 'ATID'
 'YU Torah miTzion Beit Midrash Zichron Dov' 'Ishei Yisroel' 'Chaburos'
 'Congregation Beis Avraham  Flatbush' 'HALB' 'Yeshivat Hakotel'
 'Ohr Yerushalayim' 'Naaleh' 'Beit Knesset of North Woodmere'
 'Zayit Israel Morning Kollel' 'LightLomdusLamdanut' 'HS Bekius'
 'Mishna Yomi LZN Daniel Ari ben Avraham Kadesh'
 'Motzai Shabbos Lecture Series' 'IBC Jewish Studies' 'Yadin Yadin'
 'Weekly Sichah' 'Piskei Rav Moshe' 'Praying Your Age'
 'Gruss Institute Lecture Series' 'Fair Lawn Beis Medrash'
 'Yeshivat Lev Hatorah' 'Five Minute Middah' 'Five Minute Halacha'
 'Ten Minute Mussar' 'My Community' 'Netiv Aryeh' 'Mishna B

In [3]:
# from src.pipeline.data_processor import DataProcessor, CleanedData

# def get_from_db(data:dict):
#     dp = DataProcessor()
#     query = f"""
#         SELECT * FROM "{CleanedData.CATEGORIES.value}" cat
#         JOIN "{CleanedData.SHIURIM.value}" sh ON sh.shiur = cat.shiur
#         WHERE cat."{data.get('category')}" = 1
#         AND cat."{data.get('subcategory')}" = 1
#         AND sh.series_name = '{data.get('series')}'
#     """
#     df = dp.load_query(query)
#     return df

In [4]:
from datetime import date, timedelta
import pandas as pd
import requests


def generate_calendar(start_date:date=date.today()-timedelta(days=7), end_date:date=date.today()+timedelta(days=3650)):
    parashat_mapping = {
    'Parashat Bereshit': 'subcategory_Bereishit',
    'Parashat Noach': 'subcategory_Noach',
    'Parashat Lech-Lecha': 'subcategory_Lech Lecha',
    'Parashat Vayera': 'subcategory_Vayera',
    'Parashat Chayei Sara': 'subcategory_Chayei Sara',
    'Parashat Toldot': 'subcategory_Toldot',
    'Parashat Vayetzei': 'subcategory_Vayetzei',
    'Parashat Vayishlach': 'subcategory_Vayishlach',
    'Parashat Vayeshev': 'subcategory_Vayeshev',
    'Parashat Miketz': 'subcategory_Miketz',
    'Parashat Vayigash': 'subcategory_Vayigash',
    'Parashat Vayechi': 'subcategory_Vayechi',
    'Parashat Shemot': 'subcategory_Shemot',
    'Parashat Vaera': 'subcategory_Vaera',
    'Parashat Bo': 'subcategory_Bo',
    'Parashat Beshalach': 'subcategory_Beshalach',
    'Parashat Yitro': 'subcategory_Yitro',
    'Parashat Mishpatim': 'subcategory_Mishpatim',
    'Parashat Terumah': 'subcategory_Terumah',
    'Parashat Tetzaveh': 'subcategory_Tetzaveh',
    'Parashat Ki Tisa': 'subcategory_Ki Tisa',
    'Parashat Vayakhel': 'subcategory_Vayakhel',
    'Parashat Pekudei': 'subcategory_Pekudei',
    'Parashat Vayakhel-Pekudei': 'subcategory_Vayakhel-Pekudei',
    'Parashat Vayikra': 'subcategory_Vayikra',
    'Parashat Tzav': 'subcategory_Tzav',
    'Parashat Shmini': 'subcategory_Shmini',
    'Parashat Tazria-Metzora': 'subcategory_Tazria-Metzora',
    'Parashat Tazria': 'subcategory_Tazria',
    'Parashat Metzora': 'subcategory_Metzora',
    'Parashat Achrei Mot-Kedoshim': 'subcategory_Acharei Mot-Kedoshim',
    'Parashat Achrei Mot': 'subcategory_Acharei Mot',
    'Parashat Kedoshim': 'subcategory_Kedoshim',
    'Parashat Emor': 'subcategory_Emor',
    'Parashat Behar-Bechukotai': 'subcategory_Behar-Bechukotai',
    'Parashat Behar': 'subcategory_Behar',
    'Parashat Bechukotai': 'subcategory_Bechukotai',
    'Parashat Bamidbar': 'subcategory_Bamidbar',
    'Parashat Nasso': 'subcategory_Nasso',
    'Parashat Beha’alotcha': 'subcategory_Behaalotecha',
    'Parashat Sh’lach': 'subcategory_Shelach',
    'Parashat Korach': 'subcategory_Korach',
    'Parashat Chukat': 'subcategory_Chukat',
    'Parashat Balak': 'subcategory_Balak',
    'Parashat Chukat-Balak': 'subcategory_Chukat-Balak',
    'Parashat Pinchas': 'subcategory_Pinchas',
    'Parashat Matot-Masei': 'subcategory_Matot-Masei',
    'Parashat Matot': 'subcategory_Matot',
    'Parashat Masei': 'subcategory_Masei',
    'Parashat Devarim': 'subcategory_Devarim',
    'Parashat Vaetchanan': 'subcategory_Vaetchanan',
    'Parashat Eikev': 'subcategory_Eikev',
    'Parashat Re’eh': 'subcategory_Re\'eh',
    'Parashat Shoftim': 'subcategory_Shoftim',
    'Parashat Ki Teitzei': 'subcategory_Ki Teitzei',
    'Parashat Ki Tavo': 'subcategory_Ki Tavo',
    'Parashat Nitzavim': 'subcategory_Nitzavim',
    'Parashat Vayeilech': 'subcategory_Vayeilech',
    'Parashat Nitzavim-Vayeilech': 'subcategory_Nitzavim-Vayeilech',
    'Parashat Ha’azinu': 'subcategory_Haazinu'
}
    nach_mapping = {
    'Joshua': 'subcategory_Yehoshua',
    'Judges': 'subcategory_Shoftim',
    'I Samuel': 'subcategory_Shmuel',
    'II Samuel': 'subcategory_Shmuel',
    'I Kings': 'subcategory_Melachim',
    'II Kings': 'subcategory_Melachim',
    'Isaiah': 'subcategory_Yeshayahu',
    'Jeremiah': 'subcategory_Yirmiyahu',
    'Ezekiel': 'subcategory_Yechezkel',
    'Hosea': 'subcategory_Hosheia',
    'Joel': 'subcategory_Yoel',
    'Amos': 'subcategory_Amos',
    'Obadiah': 'subcategory_Ovadia',
    'Jonah': 'subcategory_Yonah',
    'Micah': 'subcategory_Michah',
    'Nachum': 'subcategory_Nachum',
    'Habakkuk': 'subcategory_Chabakuk',
    'Zephaniah': 'subcategory_Tzefania',
    'Haggai': 'subcategory_Chaggai',
    'Zechariah': 'subcategory_Zecharia',
    'Malachi': 'subcategory_Malachi',
    'Psalms': 'subcategory_Tehillim',
    'Proverbs': 'subcategory_Mishlei',
    'Job': 'subcategory_Iyov',
    'Song of Songs': 'subcategory_Shir Hashirim',
    'Ruth': 'subcategory_Rut',
    'Lamentations': 'subcategory_Eichah',
    'Ecclesiastes': 'subcategory_Kohelet',
    'Esther': 'subcategory_Esther',
    'Daniel': 'subcategory_Daniel',
    'Ezra': 'subcategory_Ezra & Nechemia',
    'Nehemiah': 'subcategory_Ezra & Nechemia',
    'I Chronicles': 'subcategory_Divrei Hayamim',
    'II Chronicles': 'subcategory_Divrei Hayamim'
}
    masechta_mapping = {
    'Berachot': 'subcategory_Berachot',
    'Berakhot': 'subcategory_Berachot',
    'Peah': 'subcategory_Peah',
    'Demai': 'subcategory_Demai',
    'Kilayim': 'subcategory_Kilayim',
    'Sheviit': 'subcategory_Sheviit',
    'Terumot': 'subcategory_Terumot',
    'Maasrot': 'subcategory_Maasrot',
    'Maaser Sheni': 'subcategory_Maaser Sheni',
    'Challah': 'subcategory_Challah',
    'Orlah': 'subcategory_Orlah',
    'Bikkurim': 'subcategory_Bikkurim',
    'Shabbat': 'subcategory_Shabbat',
    'Eruvin': 'subcategory_Eruvin',
    'Pesachim': 'subcategory_Pesachim',
    'Shekalim': 'subcategory_Shekalim',
    'Yoma': 'subcategory_Yoma',
    'Sukkah': 'subcategory_Sukkah',
    'Beitzah': 'subcategory_Beitzah',
    'Rosh Hashana': 'subcategory_Rosh Hashana',
    'Rosh Hashanah': 'subcategory_Rosh Hashana',
    'Taanit': 'subcategory_Ta\'anit',
    'Megillah': 'subcategory_Megillah',
    'Moed Katan': 'subcategory_Moed Katan',
    'Chagigah': 'subcategory_Chagigah',
    'Yevamot': 'subcategory_Yevamot',
    'Ketubot': 'subcategory_Ketubot',
    'Nedarim': 'subcategory_Nedarim',
    'Nazir': 'subcategory_Nazir',
    'Sotah': 'subcategory_Sotah',
    'Gitin': 'subcategory_Gitin',
    'Gittin': 'subcategory_Gitin',
    'Kiddushin': 'subcategory_Kiddushin',
    'Baba Kamma': 'subcategory_Baba Kamma',
    'Baba Metzia': 'subcategory_Baba Metzia',
    'Baba Batra': 'subcategory_Baba Batra',
    'Bava Kamma': 'subcategory_Baba Kamma',
    'Bava Metzia': 'subcategory_Baba Metzia',
    'Bava Batra': 'subcategory_Baba Batra',
    'Sanhedrin': 'subcategory_Sanhedrin',
    'Makkot': 'subcategory_Makkot',
    'Shevuot': 'subcategory_Shevuot',
    'Avodah Zarah': 'subcategory_Avodah Zara',
    'Horayot': 'subcategory_Horayot',
    'Eduyot': 'subcategory_Eduyot',
    'Avot': 'subcategory_Avot',
    'Zevachim': 'subcategory_Zevachim',
    'Menachot': 'subcategory_Menachot',
    'Chullin': 'subcategory_Chullin',
    'Bechorot': 'subcategory_Bechorot',
    'Bekhorot': 'subcategory_Bechorot',
    'Arachin': 'subcategory_Arachin',
    'Arakhin': 'subcategory_Arachin',
    'Temurah': 'subcategory_Temurah',
    'Keritot': 'subcategory_Keritot',
    'Meilah': 'subcategory_Meilah',
    'Kinnim': 'subcategory_Kinnim',
    'Tamid': 'subcategory_Tamid',
    'Midot': 'subcategory_Midot',
    'Middot': 'subcategory_Midot',
    'Kelim': 'subcategory_Kelim',
    'Oholot': 'subcategory_Ohalot',
    'Negaim': 'subcategory_Negaim',
    'Parah': 'subcategory_Parah',
    'Tahorot': 'subcategory_Taharot',
    'Mikvaot': 'subcategory_Mikvaot',
    'Makhshirin': 'subcategory_Machshirin',
    'Zavim': 'subcategory_Zavim',
    'Tevul Yom': 'subcategory_Tevul Yom',
    'Yadayim': 'subcategory_Yadayim',
    'Oktzin': 'subcategory_Ukzin',
    'Niddah': 'subcategory_Niddah'
}
    holiday_mapping = {
    'Tzom Tammuz': 'subcategory_Shiva Asar b\'Tamuz',
    'Shabbat Chazon': 'subcategory_Shabbat Chazon',
    'Tish’a B’Av': 'subcategory_Tisha Bav',
    'Shabbat Nachamu': 'subcategory_Shabbat Nachamu',
    'Tu B’Av': 'subcategory_Tu B\'Av',
    'Leil Selichot': 'subcategory_Slichot',
    'Rosh Hashana': 'subcategory_Rosh Hashana',
    'Rosh Hashana II': 'subcategory_Rosh Hashana',
    'Shabbat Shuva': 'subcategory_Shabbat Shuvah',
    'Tzom Gedaliah': 'subcategory_Tzom Gedalia',
    'Yom Kippur': 'subcategory_Yom Kippur',
    'Sukkot I': 'subcategory_Sukkot',
    'Sukkot II': 'subcategory_Sukkot',
    'Sukkot III (CH’’M)': 'subcategory_Sukkot',
    'Sukkot IV (CH’’M)': 'subcategory_Sukkot',
    'Sukkot V (CH’’M)': 'subcategory_Sukkot',
    'Sukkot VI (CH’’M)': 'subcategory_Sukkot',
    'Sukkot VII (Hoshana Raba)': 'subcategory_Hoshana Rabbah',
    'Shmini Atzeret': 'subcategory_Shmini Atzeret/Simchat Torah',
    'Simchat Torah': 'subcategory_Shmini Atzeret/Simchat Torah',
    'Chanukah: 1 Candle': 'subcategory_Chanukah',
    'Chanukah: 2 Candles': 'subcategory_Chanukah',
    'Chanukah: 3 Candles': 'subcategory_Chanukah',
    'Chanukah: 4 Candles': 'subcategory_Chanukah',
    'Chanukah: 5 Candles': 'subcategory_Chanukah',
    'Chanukah: 6 Candles': 'subcategory_Chanukah',
    'Chanukah: 7 Candles': 'subcategory_Chanukah',
    'Chanukah: 8 Candles': 'subcategory_Chanukah',
    'Chanukah: 8th Day': 'subcategory_Chanukah',
    'Asara B’Tevet': 'subcategory_Assara B\'Tevet',
    'Shabbat Shirah': 'subcategory_Shabbat Shira',
    'Tu BiShvat': 'subcategory_Tu Beshvat',
    'Shabbat Shekalim': 'subcategory_Shekalim',
    'Shabbat Zachor': 'subcategory_Zachor',
    'Ta’anit Esther': 'subcategory_Taanit Esther',
    'Purim': 'subcategory_Purim',
    'Shabbat Parah': 'subcategory_Parah',
    'Shabbat HaChodesh': 'subcategory_Hachodesh',
    'Shabbat HaGadol': 'subcategory_Shabbat Hagadol',
    'Pesach I': 'subcategory_Pesach',
    'Pesach II': 'subcategory_Pesach',
    'Pesach III (CH’’M)': 'subcategory_Pesach',
    'Pesach IV (CH’’M)': 'subcategory_Pesach',
    'Pesach V (CH’’M)': 'subcategory_Pesach',
    'Pesach VI (CH’’M)': 'subcategory_Pesach',
    'Pesach VII': 'subcategory_Pesach',
    'Pesach VIII': 'subcategory_Pesach',
    'Yom HaShoah': 'subcategory_Yom Hashoah',
    'Yom HaZikaron': 'subcategory_Yom Hazikaron',
    'Yom HaAtzma’ut': 'subcategory_Yom Ha\'atzmaut',
    'Pesach Sheni': 'subcategory_Pesach Sheni',
    'Lag BaOmer': 'subcategory_Lag BaOmer',
    'Yom Yerushalayim': 'subcategory_Yom Yerushalayim',
    'Shavuot I': 'subcategory_Shavuot',
    'Shavuot II': 'subcategory_Shavuot',
    'Tu B’Av, Shabbat Nachamu': 'subcategory_Tu B\'Av,subcategory_Shabbat Nachamu',
    'Tu BiShvat, Shabbat Shirah': 'subcategory_Tu Beshvat,subcategory_Shabbat Shira',
    'Purim Katan': 'subcategory_Purim Katan',
    'Tish’a B’Av (observed)': 'subcategory_Tisha Bav'
}
    rosh_chodesh_mapping = {
    'Rosh Chodesh Av': 'subcategory_Rosh Chodesh',
    'Rosh Chodesh Elul': 'subcategory_Rosh Chodesh',
    'Rosh Chodesh Cheshvan': 'subcategory_Rosh Chodesh',
    'Rosh Chodesh Kislev': 'subcategory_Rosh Chodesh',
    'Rosh Chodesh Tevet': 'subcategory_Rosh Chodesh',
    'Rosh Chodesh Sh’vat': 'subcategory_Rosh Chodesh',
    'Rosh Chodesh Adar': 'subcategory_Rosh Chodesh',
    'Rosh Chodesh Nisan': 'subcategory_Rosh Chodesh',
    'Rosh Chodesh Iyyar': 'subcategory_Rosh Chodesh',
    'Rosh Chodesh Sivan': 'subcategory_Rosh Chodesh',
    'Rosh Chodesh Tamuz': 'subcategory_Rosh Chodesh',
    'Rosh Chodesh Adar I': 'subcategory_Rosh Chodesh',
    'Rosh Chodesh Adar II': 'subcategory_Rosh Chodesh'
}
    url = "https://www.hebcal.com/hebcal"
    params = {
        "v": "1",
        "cfg": "json",
        "maj" : "on",
        "min" : "on",
        "nx" : "on",
        "mf" : "on",
        "ss" : "on",
        "mod" : "on",
        "s" : "on",
        "F" : "on",
        "myomi" : "on",
        "nyomi" : "on",
        "yyomi" : "on",
        "dw" : "on",
        "start": start_date,
        "end": end_date
    }
    response = requests.get(url, params=params).json()["items"]
    df = pd.DataFrame(response)
    df = df[["title", "date", "category"]]
    values_to_drop = ["Yom HaAliyah", "Rosh Hashana LaBehemot", "Erev Tish’a B’Av", "Erev Rosh Hashana", "Erev Yom Kippur", "Erev Sukkot", "Sigd", "Chag HaBanot", "Erev Purim", "Purim Meshulash", "Erev Pesach", "Shushan Purim", "Shushan Purim Katan", "Erev Shavuot", "Ta’anit Bechorot", "Tu B’Av"]
    df = df[~((df['category'] == 'holiday') & (df['title'].isin(values_to_drop)))]
    df = df.pivot_table(index="date", columns="category", values="title", aggfunc=lambda x: ', '.join(x))
    df.drop(columns=['mevarchim'], inplace=True)
    df['holiday'] = df['holiday'].replace(r'Rosh Hashana \d+', 'Rosh Hashana', regex=True)
    #parashat processing
    df['parashat'] = df['parashat'].replace(parashat_mapping)
    df['parashat'] = df['parashat'].bfill()
    #nachyomi processing
    df[['n_sefer', 'n_num']] = df['nachyomi'].str.extract(r'([A-Za-z\s]+?)\s*(\d+)', expand=True)
    df.drop(columns=['nachyomi'], inplace=True)
    df['n_sefer'] = df['n_sefer'].replace(nach_mapping)
    #dafyomi processing
    df[['d_masechta', 'd_num']] = df['dafyomi'].str.extract(r'([A-Za-z\s]+?)\s*(\d+)', expand=True)
    df.drop(columns=['dafyomi'], inplace=True)
    df['d_masechta'] = df['d_masechta'].replace(masechta_mapping)
    #dafWeekly processing
    df[['dw_masechta', 'dw_num']] = df['dafWeekly'].str.extract(r'([A-Za-z\s]+?)\s*(\d+)', expand=True)
    df.drop(columns=['dafWeekly'], inplace=True)
    df['dw_masechta'] = df['dw_masechta'].replace(masechta_mapping)
    df['dw_masechta'] = df['dw_masechta'].ffill()
    df['dw_num'] = df['dw_num'].ffill()
    #yerushalmiyomi processing
    df[['y_masechta', 'y_num']] = df['yerushalmi'].str.extract(r'([A-Za-z\s]+?)\s*(\d+)', expand=True)
    df.drop(columns=['yerushalmi'], inplace=True)
    df['y_masechta'] = df['y_masechta'].replace(masechta_mapping)
    #mishnayomi processing
    df[['m_masechta', 'm_num1', 'm_num2']] = df['mishnayomi'].str.extract(r'([A-Za-z\s]+?)\s*(\d+):(\d+)', expand=True)
    df.drop(columns=['mishnayomi'], inplace=True)
    df['m_masechta'] = df['m_masechta'].replace(masechta_mapping)
    #holiday processing
    df['holiday'] = df['holiday'].replace(holiday_mapping)
    #roshchodesh processing
    df['roshchodesh'] = df['roshchodesh'].replace(rosh_chodesh_mapping)
    df.to_csv("cycles.csv")
    print(df.head())

In [5]:
generate_calendar()

category   holiday           parashat roshchodesh                n_sefer  \
date                                                                       
2024-07-14     NaN  subcategory_Balak         NaN  subcategory_Yeshayahu   
2024-07-15     NaN  subcategory_Balak         NaN  subcategory_Yeshayahu   
2024-07-16     NaN  subcategory_Balak         NaN  subcategory_Yeshayahu   
2024-07-17     NaN  subcategory_Balak         NaN  subcategory_Yeshayahu   
2024-07-18     NaN  subcategory_Balak         NaN  subcategory_Yeshayahu   

category   n_num              d_masechta d_num          dw_masechta dw_num  \
date                                                                         
2024-07-14    18  subcategory_Baba Batra    19  subcategory_Ketubot     97   
2024-07-15    19  subcategory_Baba Batra    20  subcategory_Ketubot     97   
2024-07-16    20  subcategory_Baba Batra    21  subcategory_Ketubot     97   
2024-07-17    21  subcategory_Baba Batra    22  subcategory_Ketubot     97   